In [58]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Load the data
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [59]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [60]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Классы нейронки и слоев

In [61]:

def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

def compute_cost(y_pred, y_true):
    m = y_true.shape[0]
    cost = -(1 / m) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return cost


class Layer:
    def __init__(self, input_shape: int, output_shape: int):
        self.weights = np.random.randn(input_shape, output_shape)
        self.bias = np.zeros((1, output_shape))


class NeuralNetwork:
    def __init__(self, layers, activation):
        self.layers = layers
        self.activation = activation

    def predict(self, X):
        return self.forward(X)[-1]

    def forward(self, X):
        a_hist = [X]
        for l in self.layers:
            z = np.dot(a_hist[-1], l.weights) + l.bias
            a = self.activation(z)
            a_hist.append(a)
        return a_hist

    def backward(self, y_true, learning_rate, a_hist):
        m = y_true.shape[0]
        dz = a_hist[-1] - y_true
        d_hist = []
        prev = None
        for l, a in zip(self.layers[::-1], a_hist[0:-1:-1]):
            if prev:
                lp, ap = prev 
                dz = np.dot(dz, lp.weights.T) * ap * (1 - ap)
            dw = (1 / m) * np.dot(a.T, dz)
            db = (1 / m) * np.sum(dz, axis=0, keepdims=True)
            d_hist.append((dw, db))
            prev = l, a

        for l, (dw, db) in zip(self.layers[::-1], d_hist):
            l.weights = l.weights - learning_rate * dw
            l.bias = l.bias - learning_rate * db

    def train(self, X, y_true, learning_rate, num_iterations):
        for i in tqdm(range(num_iterations)):
            a_hist = self.forward(X)
            cost = compute_cost(a_hist[-1], y_true)
            self.backward(y_true, learning_rate, a_hist)
            if i % 1000 == 0:
                print(f"Cost after iteration {i}: {cost:.4f}")


In [62]:
X_train = X_train.reshape((X_train.shape[0], (X_train.size//X_train.shape[0])))
print(X_train.shape)
print(y_train.shape)
nn = NeuralNetwork(
    [
        Layer(X_train.shape[1], 1000),
        Layer(1000, 512),
        Layer(512, 1)
    ], sigmoid)
nn.train(X_train, y_train, 0.001, 10)

(50000, 3072)
(50000, 1)


/tmp/ipykernel_21974/3211219221.py:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(num_iterations)):


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_21974/3211219221.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Cost after iteration 0: 823.4112


In [63]:
X_test = X_test.reshape((X_test.shape[0], (X_test.size//X_test.shape[0])))
# Evaluate the neural network on the test set
y_pred = nn.predict(X_test)

y_pred_e = np.round(y_pred, 2)
y_test_e = np.round(y_test, 2)
print(y_pred_e)
print(y_test_e)
accuracy = np.mean(y_pred_e == y_test_e)
print(f"Test accuracy: {accuracy}")

/tmp/ipykernel_21974/3211219221.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


[[0.89]
 [0.98]
 [0.99]
 ...
 [1.  ]
 [0.02]
 [0.96]]
[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]
Test accuracy: 0.0399
